In [1]:
#### PACKAGE IMPORTS ####

# Run this cell first to import all required packages. Do not make any imports elsewhere in the notebook

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization
from tensorflow.keras.regularizers import l2
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
import numpy as np
import matplotlib.pyplot as plt

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# If you would like to make further imports from Tensorflow, add them here

In [2]:
file = open("x_train.txt", "r")
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

x = []
y = []

read_x = open("x_train.txt", "r")
read_y = open("y_train.txt", "r")
read_z = open("z_train.txt", "r")

for i in range(int(line_count)):

    x.append([float(read_x.readline()), float(read_y.readline()), float(read_z.readline())])

read_x.close()
read_y.close()
read_z.close()

x_train = tf.constant(x[:-1])
y_train = tf.constant(x[1:])

print("Number of points: ", len(x))

Number of points:  20001


In [45]:
#### GRADED CELL ####

# Complete the following class. 
# Make sure to not change the class name or provided methods and signatures.

class MLP(tf.keras.layers.Layer):
    
    def __init__(self, num_layers, **kwargs):

        super(MLP, self).__init__(**kwargs)
        self.num_layers = num_layers


    def build(self, input_shape):
        
        
        self.L1 = tf.keras.layers.Dense(10, activation='tanh')
        self.L2 = tf.keras.layers.Dense(40, activation='tanh')
        self.L3 = tf.keras.layers.Dense(3, activation='tanh')
        
        
        
        #input_filters = input_shape[-1]
                
        #self.conv_1 = Conv2D(self.num_filters, kernel_size=self.kernel_size, padding="SAME",
         #                   activation='relu', kernel_regularizer=l2(self.l2reg_coeff), input_shape=input_shape)
        #self.bn_1 = BatchNormalization()
        #self.conv_2 = Conv2D(input_filters, kernel_size=self.kernel_size, padding="SAME",
         #                   activation='relu', kernel_regularizer=l2(self.l2reg_coeff))
        #self.bn_2 = BatchNormalization()

    def call(self, inputs, training=False):
        
        h = self.L1(inputs)
        h = self.L2(h)
        h = self.L3(h)
        
        #h = self.conv_1(inputs)
        #h = self.bn_1(h)
        #h = self.conv_2(h)
        #h = self.bn_2(h)
        
        return h #+ inputs

In [46]:
block1 = MLP(64, name='MLP1')
block2 = MLP(64, name='MLP2')

block1(tf.random.normal(x_train.shape)).shape
block2(tf.random.normal(x_train.shape)).shape

TensorShape([20000, 3])

In [49]:
#### GRADED CELL ####

# Complete the following function. 
# Make sure to not change the function name or arguments.

def get_shift_and_log_scale_resnet(input_shape, blocks):
    
    inputs = tf.keras.Input(shape=input_shape)
    h = inputs

    for block in blocks:
        h = block(h)

    shift, log_scale = h, h #tf.split(h, num_or_size_splits=2, axis=-1)
    log_scale = tf.math.tanh(log_scale)
    return Model(inputs=inputs, outputs=[shift, log_scale], name='name')

In [53]:
shift_and_log_scale = get_shift_and_log_scale_resnet((3), [block1, block2])

In [55]:
shift_and_log_scale.summary()

Model: "name"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
MLP1 (MLP)                   multiple                  603       
_________________________________________________________________
MLP2 (MLP)                   multiple                  603       
_________________________________________________________________
tf.math.tanh_4 (TFOpLambda)  (None, 3)                 0         
Total params: 1,206
Trainable params: 1,206
Non-trainable params: 0
_________________________________________________________________


In [56]:
class AffineCouplingLayer(tfb.Bijector):

    def __init__(self, shift_and_log_scale_fn, mask, **kwargs):
        super(AffineCouplingLayer, self).__init__(
            forward_min_event_ndims=1, **kwargs)
        self.shift_and_log_scale_fn = shift_and_log_scale_fn
        self.b = tf.cast(mask, tf.float32)

    def _forward(self, x):
        t, log_s = self.shift_and_log_scale_fn(x * self.b)
        y = self.b * x + (1 - self.b) * (x * tf.exp(log_s) + t)
        return y

    def _inverse(self, y):
        t, log_s = self.shift_and_log_scale_fn(y * self.b)
        x = self.b * y + (1 - self.b) * ((y - t) * tf.exp(-log_s))
        return x

In [111]:
class R(tf.keras.layers.Layer):

    def __init__(self):
        super(R, self).__init__()
        
    def call(self, inputs):
        return inputs*tf.constant([[1., -1., -1.]])

In [148]:
class RealNVPModel(tf.keras.Model):
    
    def __init__(self, shift_and_log_scale, **kwargs):
        super(RealNVPModel, self).__init__()
        
        self.R = R()
        
        affine_coupling_layer1 = AffineCouplingLayer(shift_and_log_scale, tf.constant([0.,1.,1.]))
        affine_coupling_layer2 = AffineCouplingLayer(shift_and_log_scale, tf.constant([1.,0.,1.]))
        affine_coupling_layer3 = AffineCouplingLayer(shift_and_log_scale, tf.constant([1.,1.,0.]))
        affine_coupling_layer4 = AffineCouplingLayer(shift_and_log_scale, tf.constant([1.,0.,1.]))
        affine_coupling_layer5 = AffineCouplingLayer(shift_and_log_scale, tf.constant([0.,1.,1.]))
        affine_coupling_layer6 = AffineCouplingLayer(shift_and_log_scale, tf.constant([1.,0.,0.]))

    def call(self, input_tensor):
        
        a = self.R(input_tensor)

        a = affine_coupling_layer1._inverse(input_tensor)
        a = affine_coupling_layer2._inverse(a)
        a = affine_coupling_layer3._inverse(a)
        a = affine_coupling_layer4._inverse(a)
        a = affine_coupling_layer5._inverse(a)
        a = affine_coupling_layer6._inverse(a)
        
        a = self.R(a)

        a = affine_coupling_layer6._forward(a)
        a = affine_coupling_layer5._forward(a)
        a = affine_coupling_layer4._forward(a)
        a = affine_coupling_layer3._forward(a)
        a = affine_coupling_layer2._forward(a)
        a = affine_coupling_layer1._forward(a)
        
        return a

In [151]:
model = RealNVPModel(shift_and_log_scale)

In [153]:
model(x_train)

<tf.Tensor: shape=(20000, 3), dtype=float32, numpy=
array([[-0.96728945, -2.474693  , -1.0680715 ],
       [-0.9157934 , -3.7573836 , -3.0099275 ],
       [-0.79116714,  0.49780422, -3.2913952 ],
       ...,
       [-0.8614147 ,  0.64728415, -0.33710906],
       [-0.7182631 ,  0.61435884, -0.30246106],
       [-0.59468085,  0.7729645 , -0.30507973]], dtype=float32)>

In [155]:
model.compile(
    loss= tf.keras.losses.MeanSquaredError(),
    #loss = custom_loss,
    #optimizer=tfps.optimizers.bfgs_minimize(), #clipvalue = 0.001),
    #optimizer=keras.optimizers.SGD(0.00001), #, clipvalue = 0.001),
    optimizer=tf.keras.optimizers.Adam(0.001), #, clipvalue = 0.001),
    metrics=["accuracy"],
)

In [156]:
history = model.fit(x_train, y_train, validation_split=0.1, batch_size = 1000, epochs = 100, verbose=1) #np.array(x), [np.array(y),a,a]

Epoch 1/100
18/18 [==============================] - 2s 33ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 2/100
18/18 [==============================] - 0s 11ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 3/100
18/18 [==============================] - 0s 11ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 4/100
18/18 [==============================] - 0s 11ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 5/100
18/18 [==============================] - 0s 10ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 6/100
18/18 [==============================] - 0s 11ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 7/100
18/18 [==============================] - 0s 11ms/step - loss: 8.5845 - accuracy: 0.0538 - val_loss: 8.4637 - val_accuracy: 0.0490
Epoch 